#### context switching
when switching between processess, there are a few bits of information that need to be saved:
* %rip (the register instruction pointer)
    * this is the pointer that is pointing to the next instruction that the cpu will execute

THe code section of memory is read only.

Process:  
|Process|
|-|
|Stack|
|stack contents|
|heap|
|data|
|code|
|grows in this direction (downward)|

### Registers:  
There are 16 general purpose registers in 64-bit x86 architecture.   
There are 8 general purpose registers in 32-bit x86 architecture.

%rbp(%ebp) is used as a **frame/base pointer** (Bottom of the stack).  
%rsp(%esp) is used as a **stack pointer** (Top of the stack).

### Stack:  
The main feature of operations on stack: LIFO (Last In First Out) / FILO (First In Last Out)  
Instructions that modifies stack include: **Push** and **Pop** 
* Push moves the stack pointer down 4 bits in 32bit (8 in 64bit) and sets the value of the new address the stack pointer is pointing to whatever was passed in to 
* Pop sets the value of whatever was passed in to the value of the stack pointer, then moves the stack pointer up 4 in 32bit (8 in 64bit)

### Function Calls:  
Pass control. Instructions pass control between caller and collee %rip and CPU.  
* RIP remains on the next executable line in main().
* Old Base pointer stored. Call and returns instruction, both alter stack and instruction pointer.  

Pass data: What's the data exchange between caller and callee?  
*  The return value is always put into %rax

How does x86 implement "Pass data" for function calls?


Why do the programs sometimes need to save register values before using the register?

What are caller-saved registers?

What are calle-saved registers?

When the CPU needs to switch the current process to another one, what information related to the process needs to be preserved? So next time when it becomes active it can continue from where it was "paused".

`Kernel mode`: User mode and Privilege mode.  
It will go to the privileged mode when it is distributing the processes:
* Interrupt, exceptions, and inputs (like mouse or keyboard, because they need immediate response)

```c
1. callq: Fetched by CPU. But not executed yet.
    %rip = 
    %rsp = 
2. Right after Callq is executed.
    %rip = 
    %rsp =
```

## Process
* A program in execution
* containers for programs
    * virtual memory
    * scheduling 

## Fair Share
even if a program is running forever `while(1)`, it is still taking turns inside the cpu

# Architectural Support for the OS
* Not all instructions are created equal ...
    * Non-privileged instructions
        * can affect only the current program
    * privileged instructions
        * may affect entire systems

Which instructions should be privileged?
* I/O instructions
* those that affect how memory is mapped
* halt instruction

Who is privileged?
* You are not
* only the operating-system kernel is



Entering privileged mode:
* How is OS invoked?
    * Very carefully ...
    * Strictly on response to interrupts and exceptions

Interrupts and exceptions:
* Thins don't always go smoothly
    * IO devices demand attention
    * timers expire
    * Programs demand OS services
    * programs demand storage be made accessible
    * programs have problems
* Interuprs
    * demand for attention by external spurces
* exceptions
    * programms running demand attention

Cont.
* Interrupt or exception invokes handler (in OS)
    * via interrupt and exception vector
    * one entry for each possible interrupt/exception
        * contains:
            * address of handler
        * code executed in privileged mode
    
```c
#include <unistd.h>
int main(){
    pid_t pid;
    if((pid = fork()) == 0){
        /* new process starts running here */
    }
    /* old process contrunues here */
}
```

|parent: 27342|
|-|
|pid = 27355|
|ParentPid = 27432 |
|fork() <- IP |

---
|child: 27355|
|-|
|pid = 0 |
|ParentPid = 27342|
|fork() <- code|


```c
int main(){
    pid_t pid;
    pid_t = ParentPid = getpid(); // parentPid is set for parent
    if((pid = fork() == 0{ 
        // child is created, with a parentPid being the parentPid since it is a copy
        // the child pid val is set to 0
        // the 
        printf("Child: %d, %d, %d\n", pid, ParentPid, getpid());
        // prints 0, 27342, 27355
        return 0;
    }
    printf("Parent: %d, %d, %d\n", pid, ParentPid, getpid());
    // prints  27355, 27342, 27342
    return 0;
}
// because what the program prints depends on the cpu scheduling, it's output will not be consistent
```

```c
int flag; // global val will always get a value, so it is set to 0 (but if it was a pointer it would be null)
int main(){
    while(flag ==0){
        if(fork() == 0){
            flag = 1;
            exit(0); // caise process to termiante
        }
    }
}
```
Parent is infinite, and the child is always terminated

Forks are called once and returned twice:  
returned to the parent and to the child

One Stack per mode:
|User Stack|
|-|
|Frame 1|
|Frame 2|
  

|Kernel Stack|
|-|
|Intrp/Excp Frame|
|Frame 3|
|Frame 4|

### Exceptions
An exception is a transfer of control to the OS kernel in response to some event  
Event -> `I_current` -> User Code -> **Exception** -> Kernel code -> Exceptioon processing by exception handler -> returns to `I_current`

Examples of exceptions: 
* Divide by 0
* Arithmetic overflow
* Page fault
* I/O request completes
* Ctrl-C

Eacj x84-64 system call has a unique ID number  
|Number|Name|Description|
|-|-|-|
|0|read|Read file|
|1|write|Write file|
|2|open|Open file|
|3|close|Close file|
|4|stat| Get info about file|
|57|fork|Create process|
|59|execve| Execute a program|
|60|_exit|Terminate process|
|62|kill|Send signal to process|

### System Call Example: Opening File
* User calls: open(filename, options)
* Calls _open function, which invokes system call instruction syscall

```assembly
mov $0x2, %eax # open is syscall 2
syscall # return value in %rax
cmp $0xfffffffffffffffff001, %rax # this tries to read the value of %rax, making sure that it can actually be read (not corrupt)
```
syscall passes the control, then after completing returns to the same spot  
follows the same calling convention  

Fault example: Page Fault:  
User writes to memory location  
That portion (page) of user's memory is currently on disk
```assembly
movl $0xd, 0x8049d10
```

```c
int main(){
    int value = 5;
    pid_t pid;
    if((pid = fork()) == 0){ /* child process */
        value += 15;
        printf("CHILD: value = %d\n", value);
        return 0l
    }
    else if( pid > 0){ /* parent process */
        printf("PARENT: value = %d\n", value);
        return 0;
    }
}
```

```c
int main(){
    int x = 1;
    if(fork() == 0){
        printf("p1: x=%d\n", ++x); 
        // p1; 2 (child)
    } 
    printf("p2: x=%d\n", --x); 
    // p2: 1 (child because its not in an else statement it is still executed)
    // p2: 0 (parent)
    exit(0);
}
```

```c
int main()
{ 
    int value = 5;
    pid_t pid;
    if ((pid = fork()) == 0){
        value += 15;
        printf("CHILD: value = %d\n",value);
        return 0;
    }
    else if(pid>0){
        wait(NULL); // waits for the child to finish
        // you could also pass in a specific pid_t for the parent to wait until that specific pid_t is done
        printf("PARENT: value = %d\n",value);
        return 0;
    }
}

```